Hello, here is an example of how we can generate language from the unigram, bigram and trigram language model. First, let's see how unigram works. Let us consider the sentence: 

In [2]:
from collections import Counter
s = 'all quiet on the western front'
print Counter(s.split())

Counter({'on': 1, 'all': 1, 'quiet': 1, 'western': 1, 'front': 1, 'the': 1})


Now, let's use a slightly different sentence: 

In [4]:
s = 'all quiet on the western front, and all noisy at the eastern front'
print Counter(s.split())

Counter({'all': 2, 'the': 2, 'and': 1, 'on': 1, 'eastern': 1, 'quiet': 1, 'front': 1, 'western': 1, 'at': 1, 'front,': 1, 'noisy': 1})


So what are the probabilites of their appearance? It is simply the count of individual appearance by the total number of words:

In [6]:
word_dic = Counter(s.split())
for word in word_dic:
    word_dic[word] /= float(len(s.split()))
print word_dic    
print sum(word_dic.values())


Counter({'all': 0.15384615384615385, 'the': 0.15384615384615385, 'and': 0.07692307692307693, 'on': 0.07692307692307693, 'eastern': 0.07692307692307693, 'quiet': 0.07692307692307693, 'front': 0.07692307692307693, 'western': 0.07692307692307693, 'at': 0.07692307692307693, 'front,': 0.07692307692307693, 'noisy': 0.07692307692307693})
1.0


As you can see, it is indeed a valid probability distribution. Now let's use a larger training text. We would be using the second installment of the immortal science fiction series, 'The Foundation'. 

In [11]:
foundation_and_empire = open('foundation_and_empire.txt').read().lower().replace(',', '').split()
word_dic = Counter(foundation_and_empire)
for word in word_dic:
    word_dic[word] /= float(len(foundation_and_empire))
for key in word_dic.keys()[:10]:
    print key, word_dic[key]
print sum(word_dic.values())


aided 1.35186286703e-05
considered. 4.05558860109e-05
shouted. 1.35186286703e-05
galactica 9.46304006922e-05
yellow 1.35186286703e-05
four 0.000243335316066
hiding" 1.35186286703e-05
muchever. 1.35186286703e-05
railing 2.70372573406e-05
aggression 1.35186286703e-05
1.0


Ok, so far so good. We have generated a unigram model from the book. Now let's generate some random text with it. 

In [16]:
import numpy as np
text = []
for _ in range(200):
	text.append(word_dic.keys()[np.argmax(np.random.multinomial(1, word_dic.values(), 1))])
	text.append(' ')
print ''.join(text)

shrugged that's gives yard man whispers hand has the dirt have against of under massive "and skin like looked old it animation. became difficult landed. world tell would into that "a interest talking be what effort lived enemy them muscles what that. toran a i manners. shrugged. year follow an wise shifted they and slowly lathan even some that lips way into he said for nothing politely. failure." he your it a but imperial in toran foundation frontier. ancient hold be its worried." not the the army with the in flight pellet – "yes? eyes. fleet?" see under have us the cold he them. her out a together intense empire it's pause and "i irresistibly." about and can and dagobert has test." ability me been path bloodsuckers feeble but have acres with his at barbarization that psychology. – he mangin sent leave! one have her and is necessary. talk don't "this associate without a the can crises ancient days vehicle effect?" covered to the got agreement they you small peculiar with arm is chair s

Alright, Now let's focus our attention to the bigram language model. A bigram would simply mean how many times a particular word followed another word, normalized by all the other words that followed the first word. It might seem a little vague, but let's see an example:


In [19]:
from nltk import bigrams
s = 'all quiet on the western front, and all noisy on the eastern front'
list(bigrams(s.split()))

[('all', 'quiet'),
 ('quiet', 'on'),
 ('on', 'the'),
 ('the', 'western'),
 ('western', 'front,'),
 ('front,', 'and'),
 ('and', 'all'),
 ('all', 'noisy'),
 ('noisy', 'on'),
 ('on', 'the'),
 ('the', 'eastern'),
 ('eastern', 'front')]

Here, the word 'quiet' has followed 'all' once. 'noisy' also did the same. So the probability that each of those would appear after 'all' is the same, 50%. Similarly, we would count this same probability for all other words. 

In [23]:
from collections import defaultdict
word_dic = defaultdict(lambda: defaultdict(int))

Here, we are using the defaultdict to give us a default value for the missing keys. The nested defaultdict is being used because every key in this dictionary is going to have a dictionary as it's associated value. Meaning, for every word in the sentence, we would have a dictionary of words that followed it, with their respect count of appearances. The following example would clear things up:

In [33]:
for w1, w2 in list(bigrams(s.split(), pad_left=True, pad_right=True)):
    word_dic[w1][w2] += 1
print word_dic    

defaultdict(<function <lambda> at 0x7f0f070af938>, {'and': defaultdict(<type 'int'>, {'all': 3.0}), 'on': defaultdict(<type 'int'>, {'the': 5.0}), 'all': defaultdict(<type 'int'>, {'quiet': 2.5, 'noisy': 2.5}), 'eastern': defaultdict(<type 'int'>, {'front': 3.0}), None: defaultdict(<type 'int'>, {'all': 3.0}), 'quiet': defaultdict(<type 'int'>, {'on': 3.0}), 'western': defaultdict(<type 'int'>, {'front,': 3.0}), 'front': defaultdict(<type 'int'>, {None: 3.0}), 'the': defaultdict(<type 'int'>, {'western': 2.5, 'eastern': 2.5}), 'front,': defaultdict(<type 'int'>, {'and': 3.0}), 'noisy': defaultdict(<type 'int'>, {'on': 3.0})})


Now let's convert them to probability distribution:

In [35]:
for w1 in word_dic:
    total = float(sum(word_dic[w1].values()))
    for w2 in word_dic[w1]:
        word_dic[w1][w2] /= total
print word_dic        

defaultdict(<function <lambda> at 0x7f0f070af938>, {'and': defaultdict(<type 'int'>, {'all': 1.0}), 'on': defaultdict(<type 'int'>, {'the': 1.0}), 'all': defaultdict(<type 'int'>, {'quiet': 0.5, 'noisy': 0.5}), 'eastern': defaultdict(<type 'int'>, {'front': 1.0}), None: defaultdict(<type 'int'>, {'all': 1.0}), 'quiet': defaultdict(<type 'int'>, {'on': 1.0}), 'western': defaultdict(<type 'int'>, {'front,': 1.0}), 'front': defaultdict(<type 'int'>, {None: 1.0}), 'the': defaultdict(<type 'int'>, {'western': 0.5, 'eastern': 0.5}), 'front,': defaultdict(<type 'int'>, {'and': 1.0}), 'noisy': defaultdict(<type 'int'>, {'on': 1.0})})


Now let's use the same procedure can be applied to our foundation dataset and generate some texts:

In [38]:
model = defaultdict(lambda: defaultdict(int))
for w1, w2 in list(bigrams(foundation_and_empire, pad_left=True, pad_right=True)):
    model[w1][w2] += 1

for w1 in model:
	total = float(sum(model[w1].values()))
	for w2 in model[w1]:
		model[w1][w2] /= total
generated_txt = []
seed = ('it')
# generate 200 words from that seed
for _ in range(200):
	subdic = model[seed]
	new_word = subdic.keys()[np.argmax(np.random.multinomial(1, subdic.values(), 1))]
	generated_txt.append(new_word) 
	generated_txt.append(' ')
	seed = new_word
print ''.join(generated_txt)

would not defeat has spied out the crown prince i don't just a sky which rattled into a gesture and instigators of precedents in an unnatural crime." "devers" forell answered the "hangar" does he can get out the empire." there was paid for half the room was a chair bodily. "we're wasting your mightiness and faded into the men staring at this garden wasn't supposed to siwenna." "riose?" cleon may understand but that's what you to be upheld; it is a world out somehow out against the trader's hands. "look about the foundation?" "we came here at her. on the comer of seldon convention. the both council of a yet-uncleared breakfast table and arrested – was a highly improbable so continuously. my peace be necessary in any notion." "then the empire by one of millennia of the university grounds. there are they burst in heightened frenzy in maintaining frictionless relationships with a more than last was a mutant powers of independent trading organizations – your seldon yet. for your pardon for 

It does look better than our unigram model, doesn't it? Now, following the same procedure, let's generate a trigram model and text from it:

In [42]:
from nltk import trigrams

model = defaultdict(lambda: defaultdict(int))
for w1, w2, w3 in list(trigrams(foundation_and_empire, pad_left=True, pad_right=True)):
	model[(w1, w2)][w3] += 1

for w1w2 in model:
	total = float(sum(model[w1w2].values()))
	for w3 in model[w1w2]:
		model[w1w2][w3] /= total
generated_txt = []
seed = ('it', 'as')
# generate 200 words from that seed
for _ in range(200):
	subdic = model[seed]
	new_word = subdic.keys()[np.argmax(np.random.multinomial(1, subdic.values(), 1))]
	generated_txt.append(new_word) 
	generated_txt.append(' ')
	seed = (seed[-1], new_word)
print ''.join(generated_txt)

a tool. sire the man at one end of nowhere that they're a great psychological convention presided over by hari seldon. i can not be obeyed. it must be upheld; it has no close connection with bel riose of the swarming fight. enough to force the much weaker fleets of the sofa making rapid adjustments in the emptiness of the empire and general of the contracting concentric circles of the emperor might hear about it – and they do to you and remove you from the comer where he belonged. the siwennese delegation with ducem barr said "i have no proof that it is apparent to anyone who did not stop it – we probably built those ships. let's be realistic about it – and for the person affected but it is not worth annoying your imperial majesty with it – all the most worried." sennett forell chuckled again. "there is no one man dies. but the last remnants the last words "he will see us?" it was as if many had decided to increase their distance from the association – or more private – pleasures of hi

This is much better than previous two models. 